In [ ]:
# Cell 1: Single File Download Test
import cdsapi
import os

# --- Configuration ---
# Create a dedicated folder for this test
output_dir = '../data/climate_raw/api_test/'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'era5_land_usa_1981_01_temp.grib')

# --- API Request for a Single Month ---
print("Building request for January 1981...")
request_dictionary = {
    'format': 'grib',
    'variable': [
        '2m_temperature',
    ],
    'year': '1981',
    'month': '01',
    'day': [f'{d:02d}' for d in range(1, 32)],
    'time': [f'{h:02d}' for h in range(0, 24)],
    'area': [
        50, -105, 25, -80, # North, West, South, East
    ],
}

# --- Execute Download ---
try:
    if not os.path.exists(output_file):
        c = cdsapi.Client()
        print("Submitting API request to the CDS...")
        print(f"File will be saved to: {output_file}")
        
        c.retrieve(
            'reanalysis-era5-land',
            request_dictionary,
            output_file
        )
        print("\nDownload complete!")
        print(f"File size: {os.path.getsize(output_file) / 1e6:.2f} MB")
    else:
        print(f"File already exists, skipping download: {output_file}")

except Exception as e:
    print(f"\nAn error occurred: {e}")
    print("Please double-check that the 'climarisc_env' is active and your .cdsapirc file is correct.")

In [ ]:
# Cell 2: Load and Inspect the Downloaded GRIB Files
import xarray as xr

# --- File Paths ---
# Path to the file downloaded via our Python API script
api_file_path = r'../data/climate_raw/api_test/era5_land_usa_1981_01_temp.grib'

# Path to the file you downloaded manually from the website
manual_file_path = r'../data/climate_raw/api_test/data.grib'

# --- Test 1: Open the API-downloaded file ---
print("--- Attempting to open the API-downloaded file ---")
try:
    # We must explicitly use the 'cfgrib' engine to read GRIB files
    ds_api = xr.open_dataset(api_file_path, engine='cfgrib')
    print("SUCCESS: API file loaded correctly.")
    print(ds_api)
except Exception as e:
    print(f"FAILED to load API file. Error: {e}")

print("\n" + "="*50 + "\n")

# --- Test 2: Open the manually-downloaded file ---
print("--- Attempting to open the manually-downloaded file ---")
try:
    # Use the same engine to open the manual file
    ds_manual = xr.open_dataset(manual_file_path, engine='cfgrib')
    print("SUCCESS: Manually downloaded file loaded correctly.")
    print(ds_manual)
except Exception as e:
    print(f"FAILED to load manual file. Error: {e}")

In [ ]:
# Cell 1: Single File Download Test (Final Robust Version)
import cdsapi
import os
import xarray as xr

# --- Configuration ---
output_dir = '../data/climate_raw/api_test/'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'era5_land_usa_1981_01_temp_robust.grib')

# --- API Request for a Single Month ---
print("Building ROBUST request for January 1981...")
request_dictionary = {
    'format': 'grib',
    'variable': [
        '2m_temperature',
    ],
    'year': '1981',
    'month': '01',
    'day': [f'{d:02d}' for d in range(1, 32)],
    'time': [f'{h:02d}:00' for h in range(0, 24)],
    'area': [
        50, -105, 25, -80, # North, West, South, East
    ],
    # ADDED: This is a common option to ensure the file is properly packaged
    'download_format': 'unarchived'
}

# --- Execute Download ---
try:
    if not os.path.exists(output_file):
        c = cdsapi.Client()
        print("Submitting API request to the CDS...")
        print(f"File will be saved to: {output_file}")
        
        c.retrieve(
            'reanalysis-era5-land',
            request_dictionary,
            output_file
        )
        print("\nDownload complete!")
    else:
        print(f"File already exists, skipping download: {output_file}")

    # --- Verification Step ---
    print("\n--- Verifying the downloaded file ---")
    ds_test = xr.open_dataset(output_file, engine='cfgrib')
    print("SUCCESS! The downloaded file was opened correctly.")
    print(ds_test)

except Exception as e:
    print(f"\nAn error occurred: {e}")

In [ ]:
# Cell 1: Download All Months for 1981
import cdsapi
import os
import time

# --- Configuration ---
# Create a dedicated folder for the full 1981 download
output_dir = '../data/climate_raw/api_test_1981/'
os.makedirs(output_dir, exist_ok=True)

# --- API Request Loop ---
c = cdsapi.Client()
print("Starting download for all 12 months of 1981...")

for month in range(1, 13):
    output_file = os.path.join(output_dir, f'era5_land_usa_1981_{month:02d}.grib')
    
    if os.path.exists(output_file):
        print(f"File for 1981-{month:02d} already exists. Skipping.")
        continue

    print(f"\nSubmitting request for: 1981-{month:02d}")
    
    try:
        request_dictionary = {
            'format': 'grib',
            'variable': ['2m_temperature', 'total_precipitation'],
            'year': '1981',
            'month': f'{month:02d}',
            'day': [f'{d:02d}' for d in range(1, 32)],
            'time': [f'{h:02d}' for h in range(0, 24)],
            'area': [50, -105, 25, -80],
        }
        c.retrieve('reanalysis-era5-land', request_dictionary, output_file)
        print(f"Download for 1981-{month:02d} complete!")
        time.sleep(1) # A small polite pause
    except Exception as e:
        print(f"\nAn error occurred for 1981-{month:02d}: {e}")
        continue

print("\n--- Download for 1981 finished! ---")